In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Comment in this script is follwoing doxygen style in case it is needed to export in Latex. ref: https://www.doxygen.nl/manual/docblocks.html#pythonblocks
## general path config
gen_path = '/content/drive/MyDrive/Job'
proj_name = 'Fracta_test'
proj_path = os.path.join(gen_path, proj_name) # current location
plot_path = os.path.join(proj_path, 'plots') # plot output for EDA
data_path = os.path.join(proj_path, 'data') # raw data path

# GM Data loading

In [ ]:
## load data
gm_name = 'GM2022_assets.csv'
gm_path = os.path.join(data_path, gm_name)
gm_df = pd.read_csv(gm_path, encoding = "cp932") # encoding option for reading Japanese. ref: https://docs.python.org/3/library/codecs.html#standard-encodings
gm_df.head()

,GID,管種,埋設年,Shape_LENGTH,口径,活用状況,soil_ph,mean_low_temp,soil_moisture_index
0,WM09952-D,DUC(K),S63.12,24.618317,10.0,True,7.607464,4.924143,6.355882
1,WM02595-P,CIP,S29.7,514.015780,10.0,True,7.998868,5.738295,7.589341
2,WM04638-C,CIP,S11.1,447.747983,6.0,True,8.278671,5.193168,5.677802
3,WM02974-D,DUC(G),S55.2,120.438646,14.0,True,6.409756,6.154922,3.547069
4,WM08449-D,DUC(K),H28.9,0.018670,10.0,True,7.114687,8.418888,4.167609


In [ ]:
## data shape check
gm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9876 entries, 0 to 9875
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   GID                  9876 non-null   object 
 1   管種                   9876 non-null   object 
 2   埋設年                  9609 non-null   object 
 3   Shape_LENGTH         9876 non-null   float64
 4   口径                   9556 non-null   float64
 5   活用状況                 9876 non-null   bool   
 6   soil_ph              9876 non-null   float64
 7   mean_low_temp        9876 non-null   float64
 8   soil_moisture_index  9876 non-null   float64
dtypes: bool(1), float64(5), object(3)
memory usage: 627.0+ KB


# GM Data Cleaning

In [ ]:
## converts column name to english
## this is unnecessary step. It is done for consistancy of the table
col_dict = {'GID':'native_pipe_id', '管種':'material_original',
            '埋設年':'install_year_original', '口径':'diameter', '活用状況':'usage_status'}
gm_df.rename(columns = col_dict, inplace = True)
gm_df.columns = map(str.lower, gm_df.columns) # little bit of consistancy
print(gm_df.columns)

Index(['native_pipe_id', 'material_original', 'install_year_original',
       'shape_length', 'diameter', 'usage_status', 'soil_ph', 'mean_low_temp',
       'soil_moisture_index'],
      dtype='object')


In [ ]:
## converts 活用状況 (usage status) to 撤去ステータス (abandoned)
## Since 活用状況 is oppsite of 撤去ステータス, It might not need to keep it in the table
gm_df['abandoned'] = ~gm_df['usage_status']

## converts material names to company standard
mat_dict = {'DUC(K)':'DIP', 'DUC(G)':'DIP', 'CIP':'CAS', 'PVC':'PVC', 'ACON':'AC', 'STW':'SP', 'STP':'SP'}
gm_df['material'] = gm_df['material_original'].replace(mat_dict)
"""
## alternate way to convert material names by using NumPy functions and not using the for loop
old_mat_name = np.array(['ACON', 'CIP', 'DUC(G)', 'DUC(K)', 'PVC', 'STP', 'STW'], dtype = 'object')
new_mat_name = np.array(['AC', 'CAS', 'DIP', 'DIP', 'PVC', 'SP', 'SP'], dtype = 'object')
mat_ori_np = gm_df['material_original'].to_numpy(dtype = 'object')
mat_ori_idx = np.searchsorted(old_mat_name, mat_ori_np, side = 'left')
gm_df['material'] = new_mat_name[mat_ori_idx]
"""

## converts Japanese years to Gregorians
jp_yr_dict = {'M':1867, 'T':1911, 'S':1925, 'H':1988, 'R':2018}
def jp_to_gr(jp_dt):
  if type(jp_dt) == float: return 'NULL'
  jp_yr, mon = jp_dt.split('.')
  gr_yr = int(jp_yr[1:]) + jp_yr_dict[jp_yr[:1]] - 1
  return f'{gr_yr:04d}-{int(mon):02d}-01' # simpler return than using datetime package
  #return datetime.date(gr_yr, int(mon), 1).strftime('%Y-%m-%d')
  ## we can also use '!pip install Japanera' for the conversion
gm_df['install_year'] = gm_df['install_year_original'].apply(jp_to_gr)

## converts material & material_original to numerical formats in case it needed for ML
label_encoder = LabelEncoder()
gm_df['material_original_encode'] = label_encoder.fit_transform(gm_df['material_original'])
gm_df['material_encode'] = label_encoder.fit_transform(gm_df['material'])

gm_df.head()

,native_pipe_id,material_original,install_year_original,shape_length,diameter,usage_status,soil_ph,mean_low_temp,soil_moisture_index,abandoned,material,install_year,material_original_encode,material_encode
0,WM09952-D,DUC(K),S63.12,24.618317,10.0,True,7.607464,4.924143,6.355882,False,DIP,1987-12-01,3,2
1,WM02595-P,CIP,S29.7,514.015780,10.0,True,7.998868,5.738295,7.589341,False,CAS,1953-07-01,1,1
2,WM04638-C,CIP,S11.1,447.747983,6.0,True,8.278671,5.193168,5.677802,False,CAS,1935-01-01,1,1
3,WM02974-D,DUC(G),S55.2,120.438646,14.0,True,6.409756,6.154922,3.547069,False,DIP,1979-02-01,2,2
4,WM08449-D,DUC(K),H28.9,0.018670,10.0,True,7.114687,8.418888,4.167609,False,DIP,2015-09-01,3,2


# WO Data Loading

In [ ]:
## load data
wo_name = 'WO_EXPORT.csv'
wo_path = os.path.join(data_path, wo_name)
wo_df = pd.read_csv(wo_path, encoding = "cp932") # encoding option for reading Japanese. ref: https://docs.python.org/3/library/codecs.html#standard-encodings
wo_df.head()

,紐づけ管路ID,漏水項目,修繕結果,RMINFO,受付日,登録日
0,WM02656-C,亀裂,Resolved,1,20020912.0,20221114
1,WM04742-P,第三者損傷,Resolved,1,20210618.0,20221114
2,WM00040-C,亀裂,Resolved,0,20130610.0,20221114
3,WM12614-C,ピンホール（5mm以下）,Resolved,0,20080521.0,20221114
4,WM04879-A,亀裂,Resolved,1,20150425.0,20221114


In [ ]:
## data shape check
wo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   紐づけ管路ID  340 non-null    object 
 1   漏水項目     333 non-null    object 
 2   修繕結果     340 non-null    object 
 3   RMINFO   340 non-null    int64  
 4   受付日      311 non-null    float64
 5   登録日      340 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 16.1+ KB


# WO Data Cleaning

In [ ]:
## converts column name to english
## this is unnecessary step. It is done for consistancy of the table
col_dict = {'紐づけ管路ID':'native_pipe_id', '漏水項目':'leaking_items_original',
            '修繕結果':'repair_results', '受付日':'receipt_date', '登録日':'registration_date_original'}
wo_df.rename(columns = col_dict, inplace = True)
wo_df.columns = map(str.lower, wo_df.columns) # little bit of consistancy
print(wo_df.columns)

Index(['native_pipe_id', 'leaking_items_original', 'repair_results', 'rminfo',
       'receipt_date', 'registration_date_original'],
      dtype='object')


In [ ]:
## converts 受付日 to break_date
nan_idx = np.isnan(wo_df['receipt_date'])
wo_df['break_date'] = pd.to_datetime(wo_df['receipt_date'], format='%Y%m%d').astype(str)
wo_df['break_date'][nan_idx] = 'NULL'

## converts 登録日 to date string. Could converts to datetime format. Since break_date is string format, I will follow the string format
wo_df['registration_date'] = pd.to_datetime(wo_df['registration_date_original'], format='%Y%m%d').astype(str)

## converts 漏水項目 to english and numerical format in case it needed for ML
item_dict = {'亀裂':'crack', '第三者損傷':'third_party_damage', 'ピンホール（5mm以下）':'pinhole_less_than_5mm',
            '継手破損':'joint_damage', 'ピンホール（5mm以上）':'pinhole_more_than_5mm', '不明':'unknown'}
wo_df['leaking_items'] = wo_df['leaking_items_original'].replace(item_dict)
wo_df['leaking_items_encode'] = label_encoder.fit_transform(wo_df['leaking_items'])

## converts 修繕結果 to numerical format in case it needed for ML
wo_df['repair_results_encode'] = label_encoder.fit_transform(wo_df['repair_results'])

## To follow this instruction, 漏水事故は自然に発生した漏水に絞る必要がある。(モデリングの観点からは、入力変数とターゲット変数の相関関係があるのが前提)。,
## All the rows that corresponeded to '第三者損傷':'third_party_damage' needed to be removed from the table.
## Let just keep the index of the 'third_party_damage' for now. I will remove them after EDA or ML by reconstruction
wo_tpd_idx = wo_df['leaking_items'] == 'third_party_damage'

wo_df.head()

<ipython-input-257-a903e6df4d95>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wo_df['break_date'][nan_idx] = 'NULL'


,native_pipe_id,leaking_items_original,repair_results,rminfo,receipt_date,registration_date_original,break_date,registration_date,leaking_items,leaking_items_encode,repair_results_encode
0,WM02656-C,亀裂,Resolved,1,20020912.0,20221114,2002-09-12,2022-11-14,crack,0,0
1,WM04742-P,第三者損傷,Resolved,1,20210618.0,20221114,2021-06-18,2022-11-14,third_party_damage,4,0
2,WM00040-C,亀裂,Resolved,0,20130610.0,20221114,2013-06-10,2022-11-14,crack,0,0
3,WM12614-C,ピンホール（5mm以下）,Resolved,0,20080521.0,20221114,2008-05-21,2022-11-14,pinhole_less_than_5mm,2,0
4,WM04879-A,亀裂,Resolved,1,20150425.0,20221114,2015-04-25,2022-11-14,crack,0,0


# EDA for sanity checking and filling missing data

In [ ]:
np.char.join(' - ', 'osd')

array('o - s - d', dtype='<U9')

In [ ]:
np.char.replace(np.asarray(gm_df['install_year_original'], dtype = str), '.', '-')

array(['S63-12', 'S29-7', 'S11-1', ..., 'H14-7', 'T7-2', 'S22-8'],
      dtype='<U6')

In [ ]:
gm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9876 entries, 0 to 9875
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   native_pipe_id         9876 non-null   object 
 1   material_original      9876 non-null   object 
 2   install_year_original  9609 non-null   object 
 3   shape_length           9876 non-null   float64
 4   diameter               9556 non-null   float64
 5   usage_status           9876 non-null   bool   
 6   soil_ph                9876 non-null   float64
 7   mean_low_temp          9876 non-null   float64
 8   soil_moisture_index    9876 non-null   float64
 9   abandoned              9876 non-null   bool   
 10  material               9876 non-null   object 
 11  install_year           9876 non-null   object 
dtypes: bool(2), float64(5), object(5)
memory usage: 791.0+ KB


In [ ]:
!pip install Japanera

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/util

In [ ]:
gm_df['install_year'][0]

'1987-12-01'

In [ ]:
from japanera import EraDate

# The library outputs dates corresponding to all eras starting with "H",
# as it's somewhat ambiguous which is being referred to
date_candidates = EraDate.strptime('S63.12', '%-h%-y.%m')
date_candidates
# Use max(), with the assumption that the most recent era is the correct one
#max(date_candidates)

[EraDate(63, 12, 1, Era('西暦', 'Seireki', datetime.date(1, 1, 1), None, <EraType.COMMON: 'common'>)),
 EraDate(1988, 12, 1, Era('昭和', 'Shouwa', datetime.date(1926, 12, 25), datetime.date(1989, 1, 8), <EraType.GENERAL: 'general'>))]

In [ ]:
import datetime as dt
def parse_heisei(date_string, sep='.'):
    y, m, d = date_string.split(sep)
    return dt.date(year=1989 + int(y[1:]) - 1, month=int(m), day=int(d))
gm_df.head()

,native_pipe_id,material_original,install_year_original,shape_length,diameter,usage_status,soil_ph,mean_low_temp,soil_moisture_index,abandoned,material,install_year
0,WM09952-D,DUC(K),S63.12,24.618317,10.0,True,7.607464,4.924143,6.355882,False,DIP,1987-12-01
1,WM02595-P,CIP,S29.7,514.015780,10.0,True,7.998868,5.738295,7.589341,False,CAS,1953-07-01
2,WM04638-C,CIP,S11.1,447.747983,6.0,True,8.278671,5.193168,5.677802,False,CAS,1935-01-01
3,WM02974-D,DUC(G),S55.2,120.438646,14.0,True,6.409756,6.154922,3.547069,False,DIP,1979-02-01
4,WM08449-D,DUC(K),H28.9,0.018670,10.0,True,7.114687,8.418888,4.167609,False,DIP,2015-09-01


In [ ]:
import datetime

In [ ]:
datetime.date(1988, 12, 1)

datetime.date(1988, 12, 1)

In [ ]:
gm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9876 entries, 0 to 9875
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   native_pipe_id         9876 non-null   object 
 1   material_original      9876 non-null   object 
 2   install_year_original  9609 non-null   object 
 3   shape_length           9876 non-null   float64
 4   diameter               9556 non-null   float64
 5   usage_status           9876 non-null   bool   
 6   soil_ph                9876 non-null   float64
 7   mean_low_temp          9876 non-null   float64
 8   soil_moisture_index    9876 non-null   float64
 9   abandoned              9876 non-null   bool   
 10  material               9876 non-null   object 
 11  install_year           9876 non-null   object 
dtypes: bool(2), float64(5), object(5)
memory usage: 791.0+ KB
